In [1]:
import pandas as pd
# from datasets import load_dataset
from PIL import Image
import numpy as np
import time
# import openai
import os

In [ ]:
df = pd.read_csv("diffusion_db_unaltered.csv") 

In [3]:

'''dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)
df=pd.DataFrame(dataset)
df.head()
'''

"dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)\ndf=pd.DataFrame(dataset)\ndf.head()\n"

In [5]:

''' dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True) '''

" dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True) "

In [6]:

batch_size = 10000
df_list = []  # List to store DataFrame batches

# Process dataset in batches
for i in range(0, len(dataset['train']), batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))
    df_batch = batch.to_pandas()
    df_list.append(df_batch)  # Append batch to list

# Concatenate all batches into a final DataFrame
prompts = pd.concat(df_list, ignore_index=True)
prompts.info()
# Display the first few rows
prompts.head()


NameError: name 'dataset' is not defined

In [ ]:
image_path = prompts.loc[0, 'image']['path']  # Extract the image path
image = Image.open(image_path)
image.show()
image_array = np.array(image)
print(image_array.shape)

In [ ]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [ ]:
print(image_array)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")
# Initialize the OpenAI client
client = openai

In [ ]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [ ]:
import re

def numberify(word):
    # Define the user prompt
    prompt = f"HERE IS THE WORD: {word}"

    try:
        # First classification: 4-digit number
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number based on the following criteria.
    The number will be 4 digits. The first 4 digits determine how 'bad' the word may be.
    - Filler words = 1111
    - Non-explicit subjects = 2222
    - Non-explicit verbs = 3333
    - Explicit words increase progressively (e.g., 5555, 6666, 7777, 8888)
    - Use 9999 only for extreme cases.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        classification = response.choices[0].message.content.strip()

        # Second classification: 2-digit number based on implication
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number from 00 to 99 based on severity.
    - Use a balanced scale with 99 reserved for extreme words.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        secondary = response.choices[0].message.content.strip()

        # Third classification: Letter count in 2-digit format
        letter_count = len(re.findall(r'[a-zA-Z]', word))
        formatted_count = str(letter_count).zfill(2)  # Ensures 2-digit format

        # Combine all values into a single 8-digit number
        final_number = f"{classification}{formatted_count}{secondary}"

        return final_number, word  # Returns the combined number and word
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1, word  # Returns -1 on error



In [ ]:
df.info()

In [ ]:
import re

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in prompts.iterrows():
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.1)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)


In [ ]:
print(df['Classification'].unique())

In [ ]:
df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])


In [ ]:
print(df.loc[df['Classification'] == -1])

In [ ]:
df.loc[df['Classification'] == -1, 'Classification'] = 0

In [ ]:
print(df.loc[df['Classification'] == 2])

In [ ]:
df.to_csv('WordList.csv', index=False)


In [ ]:
"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""

In [ ]:
SafePrompt='Generate an image of a child looking to the starry night'
NSFWPrompt='Generate an image of a busty looking female with big boobs'

In [ ]:
prompts.head()

In [ ]:
prompts['allowed'] = (prompts['prompt_nsfw'] < 0.5).astype(int)

In [ ]:
prompts.head()

In [ ]:
print(prompts['allowed'].unique())

In [ ]:
prompts.info()

In [ ]:
import re

for idx, row in prompts.head(5).iterrows():
    Classprompt = row['prompt']
    Numprompt = row['prompt']

    Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
    Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt) 
    for word in re.findall(r'\b\w+\b', Classprompt):  
        if word not in df['Word'].values:
            classification_value = classify_word(word)
            numeric_value, interpreted_word = numberify(word)
        else:
            classification_value = df.loc[df['Word'] == word, 'Classification'].values[0]
            numeric_value = df.loc[df['Word'] == word, 'Number'].values[0]

        Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
        Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

    prompts.at[idx, 'Classprompt'] = str(Classprompt) 
    prompts.at[idx, 'Numprompt'] = str(Numprompt) 


In [ ]:
prompts.head()

In [ ]:
prompts.info()

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(['allowed'], ['Guess allowed'], labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
print(recall)